# Feedback Loop in RAG

이 노트북에서는 시간이 지남에 따라 지속적으로 개선되는 피드백 루프 메커니즘을 갖춘 RAG 시스템을 구현하겠습니다. 사용자 피드백을 수집하고 반영함으로써, 우리 시스템은 매 상호작용마다 더 관련성 있고 품질 높은 응답을 제공하는 법을 배웁니다.

전통적인 RAG 시스템은 정적이어서, 임베딩 유사도에만 기반해 정보를 찾아줍니다. 하지만 피드백 루프를 도입하면, 시스템은 이렇게 달라집니다:

1. 잘 작동했던 방식과 그렇지 않았던 방식을 기억하고
2. 시간이 지날수록 문서의 중요도를 조정하며
3. 효과적인 질문과 답변 쌍을 지식으로 쌓고
4.사용자와의 상호작용을 통해 점점 더 똑똑해집니다.

## 환경 설정하기
필요한 라이브러리를 가져옵니다.

In [ ]:
import os
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv
from datetime import datetime

# .env 파일 로드
load_dotenv()

API_KEY = os.environ.get('OPENAI_API_KEY')

## PDF 파일에서 텍스트 추출하기

In [ ]:
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    # API 키 설정
    genai.configure(api_key=gemini_API_KEY)
    client = genai.GenerativeModel('gemini-2.0-flash-lite')

    # PDF 파일 업로드
    with open(pdf_path, "rb") as file:
        file_data = file.read()


    prompt = "Extract all text from the provided PDF file."
    response = client.generate_content([
        {"mime_type": "application/pdf", "data": file_data},
        prompt
    ],generation_config={
            "max_output_tokens": 8192  # 최대 출력 토큰 수 설정 (예: 8192 토큰, 약 24,000~32,000자)
    })
    return response.text
    # Open the PDF file
    mypdf = fitz.open(pdf_path)
    all_text = ""  # Initialize an empty string to store the extracted text

    # Iterate through each page in the PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # Get the page
        text = page.get_text("text")  # Extract text from the page
        all_text += text  # Append the extracted text to the all_text string

    return all_text  # Return the extracted text

In [ ]:
# 이미 text 파일로 저장되어 있다면 load_text_file 함수를 사용하면 됩니다.
def load_text_file(pdf_path):

    # text 파일 로드
    with open(pdf_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()

    return text

txt_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

extracted_text = load_text_file(txt_path)
print(extracted_text[:500])

## 추출된 텍스트 chunking
pdf 에서 추출된 텍스트를 얻은 후, 검색 정확성을 향상시키기 위해 이를 더 작고 겹치는 청크로 나눕니다.

In [ ]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 겹치는 n개의 문자 세그먼트로 청크합니다.

    Args:
    text (str): 청크할 텍스트입니다.
    n (int): 각 청크의 문자 수입니다.
    overlap (int): 청크 간 겹치는 문자 수입니다.

    Returns:
    List[str]: 청크된 텍스트 리스트입니다.
    """
    chunks = []  # 청크된 텍스트를 저장할 빈 리스트를 초기화합니다.
    
    # (n - overlap) 단계로 텍스트를 반복합니다.
    for i in range(0, len(text), n - overlap):
        # 인덱스 i부터 i + n까지의 텍스트를 청크 리스트에 추가합니다.
        chunks.append(text[i:i + n])

    return chunks  

# OpenAI API 클라이언트 설정하기

In [ ]:
from openai import OpenAI

client_openai = OpenAI(api_key = API_KEY)

## Simple Vector Store 구축
NumPy를 사용하여 간단한 Vecotr store 구축

In [ ]:
class SimpleVectorStore:
    """
    NumPy를 사용하여 간단한 Vecotr store 구축
    """
    def __init__(self):
        """
        벡터 저장소 초기화
        """
        self.vectors = []
        self.texts = []
        self.metadata = []
    
    def add_item(self, text, embedding, metadata=None):
        """
        벡터 저장소에 항목 추가

        Args:
        text (str): 원본 텍스트.
        embedding (List[float]): 임베딩 벡터.
        metadata (dict, optional): 추가 메타데이터.
        """
        self.vectors.append(np.array(embedding))
        self.texts.append(text)
        self.metadata.append(metadata or {})
    
    def similarity_search(self, query_embedding, k=5, filter_func=None):
        """
        쿼리 임베딩과 가장 비슷한 항목들을 코사인 유사도를 이용해 찾는 함수

        Args:
            query_embedding (List[float]): 쿼리 임베딩 벡터.
            k (int): 반환할 결과의 수.
            filter_func (callable, optional): 메타데이터를 기반으로 결과를 필터링하는 함수.
                                            메타데이터 딕셔너리를 입력으로 받아 boolean을 반환.

        Returns:
            List[Dict]: 상위 k개 유사 항목, 각 항목은 다음을 포함:
                - text: 원본 텍스트
                - metadata: 연관된 메타데이터
                - similarity: 코사인 유사도 점수
                - relevance_score: 메타데이터 기반 관련성 또는 계산된 유사도
                
        Note: 벡터가 저장되어 있지 않거나 필터를 통과하지 못하면 빈 리스트를 반환.
        """
        if not self.vectors:
            return []  # 벡터 저장소가 비어있으면 빈 리스트 반환
        
        # 쿼리 임베딩을 numpy 배열로 변환
        query_vector = np.array(query_embedding)
        
        # 쿼리와 각 저장된 벡터 사이의 코사인 유사도 계산
        similarities = []
        for i, vector in enumerate(self.vectors):
            # 필터 기준을 통과하지 못하는 항목 건너뛰기
            if filter_func and not filter_func(self.metadata[i]):
                continue
                
            # 코사인 유사도 계산: dot product / (norm1 * norm2)
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))  # 인덱스와 유사도 점수 저장
        
        # 유사도 점수에 따라 정렬 (내림차순)
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 상위 k개 결과 딕셔너리 생성
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": score,
                # 메타데이터에서 이미 존재하는 관련성 점수가 있으면 사용, 없으면 유사도 점수 사용
                "relevance_score": self.metadata[idx].get("relevance_score", score)
            })
        
        return results

## 임베딩 생성

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
def create_embeddings(embedding_model, texts, device='cuda', batch_size=16):
    """
    SentenceTransformer 모델을 사용하여 지정된 텍스트에 대한 임베딩을 생성합니다.

    Args:
        embedding_model: 임베딩을 생성할 SentenceTransformer 모델입니다.
        texts (list): 임베딩을 생성할 입력 텍스트 리스트입니다.
        device (str): 모델을 실행할 장치 ('cuda' for GPU, 'cpu' for CPU).
        batch_size (int): 인코딩을 위한 배치 크기입니다.

    Returns:
        np.ndarray: 모델에 의해 생성된 임베딩입니다.
    """
    # 모델이 지정된 장치에 있는지 확인합니다.
    embedding_model = embedding_model.to(device)
    
    # 지정된 배치 크기로 임베딩을 생성합니다.
    embeddings = embedding_model.encode(
        texts,
        device=device,
        batch_size=batch_size,  # 메모리 사용량을 줄이기 위해 더 작은 배치 크기를 사용합니다.
        show_progress_bar=True  # 인코딩 진행 상태를 모니터링하기 위한 진행 표시줄을 표시합니다.
    )
    
    return embeddings

# GPU 사용 가능 여부를 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'mps'
print(f"Using device: {device}")

# 모델을 로드합니다.
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)

## Feedback System Functions
이제 핵심 피드백 시스템 구성 요소들을 구현해보겠습니다.

In [ ]:
def get_user_feedback(query, response, relevance, quality, comments=""):
    """
    사용자 피드백을 딕셔너리 형태로 포맷팅.
    
    Args:
        query (str): 사용자 쿼리
        response (str): 시스템 응답
        relevance (int): 관련성 점수 (1-5)
        quality (int): 품질 점수 (1-5)
        comments (str): 선택적 피드백 코멘트
        
    Returns:
        Dict: 포맷팅된 피드백
    """
    return {
        "query": query,
        "response": response,
        "relevance": int(relevance),
        "quality": int(quality),
        "comments": comments,
        "timestamp": datetime.now().isoformat()
    }

In [ ]:
import json
def store_feedback(feedback, feedback_file="feedback_data.json"):
    """
    피드백을 JSON 파일에 저장.
    
    Args:
        feedback (Dict): 피드백 데이터
        feedback_file (str): 피드백 파일 경로
    """
    with open(feedback_file, "a") as f:
        json.dump(feedback, f)
        f.write("\n")

In [ ]:
import json
def load_feedback_data(feedback_file="feedback_data.json"):
    """
    피드백 데이터를 파일에서 로드.
    
    Args:
        feedback_file (str): feedback file 경로
        
    Returns:
        List[Dict]: 피드백 항목 리스트
    """
    feedback_data = []
    try:
        with open(feedback_file, "r") as f:
            for line in f:
                if line.strip():
                    feedback_data.append(json.loads(line.strip()))
    except FileNotFoundError:
        print("No feedback data file found. Starting with empty feedback.")
    
    return feedback_data

## Document Processing with Feedback Awareness

In [ ]:
def process_document(file_path, chunk_size=1000, chunk_overlap=200):
    """
    피드백 루프를 포함한 RAG(검색 기반 생성)용 문서 처리

    이 함수는 전체 문서 처리 과정을 담당합니다.
    1. PDF에서 텍스트 추출
    2. 텍스트를 겹치게 청크로 분할
    3. 각 청크에 대한 임베딩 생성
    4. 메타데이터와 함께 벡터 데이터베이스에 저장

    Args:
    file_path (str): 파일 경로
    chunk_size (int): 각 텍스트 청크의 크기(문자)
    chunk_overlap (int): 연속된 청크 간 중복 범위(문자)

    Returns:
    Tuple[List[str], SimpleVectorStore]: A tuple containing:
        - 문서 청크 리스트
        - 임베딩과 메타데이터가 포함된 벡터 저장소
    """
    # 1. PDF에서 텍스트 추출
    #print("Extracting text from PDF...")
    #extracted_text = extract_text_from_pdf(pdf_path)

    # 1. 텍스트 파일 로드
    extracted_text = load_text_file(file_path)
    
    # 2. 청크로 분할
    print("Chunking text...")
    chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"Created {len(chunks)} text chunks")
    
    # 3. 각 청크에 대한 임베딩 생성
    print("Creating embeddings for chunks...")
    chunk_embeddings = create_embeddings(embedding_model, chunks, device=device, batch_size=1)
    
    # 4. 벡터 저장소 초기화
    store = SimpleVectorStore()
    
    # 5. 각 청크와 임베딩을 벡터 저장소에 추가
    # 피드백 기반 개선을 위한 메타데이터 포함
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        store.add_item(
            text=chunk,
            embedding=embedding,
            metadata={
                "index": i,                # 원본 문서에서의 위치
                "source": file_path,        # 원본 문서 경로
                "relevance_score": 1.0,    # 초기 관련성 점수 (피드백에 따라 업데이트됨)
                "feedback_count": 0        # 이 청크에 대해 받은 피드백 수
            }
        )
    
    print(f"Added {len(chunks)} chunks to the vector store")
    return chunks, store

## Relevance Adjustment Based on Feedback

In [ ]:
def assess_feedback_relevance(query, doc_text, feedback):
    """
    LLM을 활용해 이전 피드백이 현재 쿼리와 문서에 관련 있는지 평가합니다.
    
    이 함수는 현재 검색에 어떤 과거 피드백이 영향을 미쳐야 하는지 판단하는 데 도움을 줍니다.
    
    현재 쿼리, 과거 쿼리와 피드백, 그리고 문서 내용을 LLM에 전달해 관련성을 평가받습니다.
    
    Args:
        query (str): 현재 사용자 쿼리
        doc_text (str): 평가할 문서의 텍스트 내용
        feedback (Dict): 'query'와 'response' 키가 포함된 이전 피드백 데이터
        
    Returns:
        bool: 피드백이 현재 쿼리와 문서와 관련 있는지 여부
    """
    # 시스템 프롬프트 정의
    system_prompt = """You are an AI system that determines if a past feedback is relevant to a current query and document.
    Answer with ONLY 'yes' or 'no'. Your job is strictly to determine relevance, not to provide explanations."""

    # 사용자 프롬프트 생성
    user_prompt = f"""
    Current query: {query}
    Past query that received feedback: {feedback['query']}
    Document content: {doc_text[:500]}... [truncated]
    Past response that received feedback: {feedback['response'][:500]}... [truncated]

    Is this past feedback relevant to the current query and document? (yes/no)
    """

    # LLM API 호출
    response = client_openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0  
    )
    
    # 응답 추출 및 정규화
    answer = response.choices[0].message.content.strip().lower()
    return 'yes' in answer  # 'yes'가 포함되어 있으면 True 반환

In [ ]:
def adjust_relevance_scores(query, results, feedback_data):
    """
    과거 피드백을 반영해 문서의 관련성 점수를 조정하여 검색 품질을 향상시킵니다.
    
    이 함수는 이전 사용자 피드백을 분석해, 현재 쿼리와 관련된 피드백을 찾아내고
    피드백의 관련성 평가에 따라 점수 보정값을 계산한 뒤, 결과를 다시 정렬합니다.
    
    Args:
        query (str): 현재 사용자 쿼리
        results (List[Dict]): 원본 유사도 점수가 포함된 검색된 문서
        feedback_data (List[Dict]): 사용자 평가가 포함된 과거 피드백
        
    Returns:
        List[Dict]: 관련성 점수가 조정된 결과, 새로운 점수에 따라 정렬됨
    """
    # 피드백 데이터가 없으면 원본 결과 반환
    if not feedback_data:
        return results
    
    print("Adjusting relevance scores based on feedback history...")
    
    # Process each retrieved document
    for i, result in enumerate(results):
        document_text = result["text"]
        relevant_feedback = []
        
        # 이 특정 문서와 쿼리 조합에 대한 관련성 있는 피드백 찾기
        # LLM을 사용하여 각 과거 피드백 항목의 관련성 평가
        for feedback in feedback_data:
            is_relevant = assess_feedback_relevance(query, document_text, feedback)
            if is_relevant:
                relevant_feedback.append(feedback)
        
        # 관련성 있는 피드백이 있으면 점수 조정
        if relevant_feedback:
            # 모든 적용 가능한 피드백 항목에서 평균 관련성 등급 계산
            # 피드백 관련성은 1-5 범위 (1=관련 없음, 5=매우 관련 있음)
            avg_relevance = sum(f['relevance'] for f in relevant_feedback) / len(relevant_feedback)
            
            # 평균 관련성을 0.5-1.5 범위의 점수 수정자로 변환
            # - 3/5 미만의 점수는 원래 유사도를 감소시킴 (수정자 < 1.0)
            # - 3/5 이상의 점수는 원래 유사도를 증가시킴 (수정자 > 1.0)
            modifier = 0.5 + (avg_relevance / 5.0)
            
            # 원래 유사도에 수정자 적용
            original_score = result["similarity"]
            adjusted_score = original_score * modifier
            
            # 결과 딕셔너리에 새로운 점수와 피드백 메타데이터 업데이트
            result["original_similarity"] = original_score  # 원래 유사도 유지
            result["similarity"] = adjusted_score           # 기본 점수 업데이트
            result["relevance_score"] = adjusted_score      # 관련성 점수 업데이트
            result["feedback_applied"] = True               # 피드백 적용 플래그
            result["feedback_count"] = len(relevant_feedback)  # 사용된 피드백 항목 수
            
            # 조정에 대한 세부 정보 logging
            print(f"  Document {i+1}: Adjusted score from {original_score:.4f} to {adjusted_score:.4f} based on {len(relevant_feedback)} feedback(s)")
    
    # 조정된 점수에 따라 결과 다시 정렬
    results.sort(key=lambda x: x["similarity"], reverse=True)
    
    return results

## Fine-tuning Our Index with Feedback

In [ ]:
def fine_tune_index(current_store, chunks, feedback_data):
    """
    고품질 피드백을 사용하여 검색 품질을 향상시키기 위해 벡터 저장소를 개선합니다.
    
    이 함수는 다음과 같은 연속적인 학습 과정을 구현합니다:
    1. 고품질 피드백 (높은 평가 점수의 Q&A 쌍) 식별
    2. 성공적인 상호작용에서 새로운 검색 항목 생성
    3. 관련성 가중치가 증가된 벡터 저장소에 추가
    
    Args:
        current_store (SimpleVectorStore): 원본 문서 청크가 포함된 현재 벡터 저장소
        chunks (List[str]): 원본 문서 텍스트 청크
        feedback_data (List[Dict]): 관련성과 품질 등급이 포함된 과거 사용자 피드백
        
    Returns:
        SimpleVectorStore: 원본 청크와 피드백 기반 콘텐츠가 포함된 개선된 벡터 저장소
    """
    print("Fine-tuning index with high-quality feedback...")
    
    # 고품질 응답만 필터링 (관련성과 품질 등급이 4 또는 5)
    # 이는 가장 성공적인 상호작용만 학습하도록 보장합니다
    good_feedback = [f for f in feedback_data if f['relevance'] >= 4 and f['quality'] >= 4]
    
    if not good_feedback:
        print("No high-quality feedback found for fine-tuning.")
        return current_store  # 좋은 피드백이 없으면 원본 저장소 반환
    
    # 원본 청크와 피드백 기반 콘텐츠가 포함된 새로운 저장소 초기화
    new_store = SimpleVectorStore()
    
    # 먼저 원본 문서 청크와 기존 메타데이터를 전송
    for i in range(len(current_store.texts)):
        new_store.add_item(
            text=current_store.texts[i],
            embedding=current_store.vectors[i],
            metadata=current_store.metadata[i].copy()  # 참조 문제를 방지하기 위해 복사 사용
        )
    
    # 좋은 피드백에서 향상된 콘텐츠 생성 및 추가
    for feedback in good_feedback:
        # 질문과 고품질 답변을 결합하는 새로운 문서 포맷팅
        # 이는 사용자 쿼리에 직접 주소를 지정하는 검색 가능한 콘텐츠를 생성합니다
        enhanced_text = f"Question: {feedback['query']}\nAnswer: {feedback['response']}"
        
        # 이 새로운 합성 문서에 대한 임베딩 벡터 생성
        embedding = create_embeddings(enhanced_text)
        
        # 원본 출처와 중요성을 식별하는 특수 메타데이터로 벡터 저장소에 추가
        new_store.add_item(
            text=enhanced_text,
            embedding=embedding,
            metadata={
                "type": "feedback_enhanced",  # 피드백에서 파생됨
                "query": feedback["query"],   # 원본 쿼리 저장
                "relevance_score": 1.2,       # 초기 관련성 증가
                "feedback_count": 1,          # 피드백 통합 추적
                "original_feedback": feedback # 완전한 피드백 레코드 유지
            }
        )
        
        print(f"Added enhanced content from feedback: {feedback['query'][:50]}...")
    
    # 개선에 대한 요약 통계 logging
    print(f"Fine-tuned index now has {len(new_store.texts)} items (original: {len(chunks)})")
    return new_store

## Complete RAG Pipeline with Feedback Loop

In [ ]:
def generate_response(query, context, model_name='gpt-4.1-mini'):

    # AI 어시스턴트의 시스템 프롬프트를 정의합니다.
    system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"
    
    user_prompt = f"""
        Context:
        {context}

        Question: {query}

        Please answer the question based only on the context provided above. Be concise and accurate.
    """
    
    response = client_openai.chat.completions.create(
        model=model_name,
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return response.choices[0].message.content

In [ ]:
def rag_with_feedback_loop(query, vector_store, feedback_data, k=5, model_name="gpt-4.1-mini"):
    """
    피드백 루프를 포함한 완전한 RAG 파이프라인을 구현합니다.    
    
    Args:
        query (str): 사용자 쿼리
        vector_store (SimpleVectorStore): 문서 청크가 포함된 벡터 저장소
        feedback_data (List[Dict]): 피드백 기록
        k (int): 검색할 문서 수
        model (str): 응답 생성을 위한 LLM 모델
        
    Returns:
        Dict: 쿼리, 검색된 문서, 응답이 포함된 결과
    """
    print(f"\n=== Processing query with feedback-enhanced RAG ===")
    print(f"Query: {query}")
    
    # 1. 쿼리 임베딩 생성
    query_embedding = create_embeddings(embedding_model, [query], device=device, batch_size=1)[0]
    
    # 2. 쿼리 임베딩을 기반으로 초기 검색 수행
    results = vector_store.similarity_search(query_embedding, k=k)
    
    # 3. 피드백을 기반으로 검색된 문서의 관련성 점수 조정
    adjusted_results = adjust_relevance_scores(query, results, feedback_data)
    
    # 4. 조정된 결과에서 텍스트 추출하여 컨텍스트 생성
    retrieved_texts = [result["text"] for result in adjusted_results]
    
    # 5. 검색된 텍스트를 연결하여 응답 생성을 위한 컨텍스트 생성
    context = "\n\n---\n\n".join(retrieved_texts)
    
    # 6. 컨텍스트와 쿼리를 사용하여 응답 생성
    print("Generating response...")
    response = generate_response(query, context, model_name)
    
    # 7. 최종 결과 컴파일
    result = {
        "query": query,
        "retrieved_documents": adjusted_results,
        "response": response
    }
    
    print("\n=== Response ===")
    print(response)
    
    return result

## Complete Workflow: From Initial Setup to Feedback Collection

In [ ]:
def full_rag_workflow(file_path, query, feedback_data=None, feedback_file="feedback_data.json", fine_tune=False):
    """
    피드백 통합을 포함한 완전한 RAG 워크플로우를 실행합니다.
    
    이 함수는 전체 RAG(검색 기반 생성) 과정을 총괄합니다:
    1. 과거 피드백 데이터를 불러옵니다
    2. 문서를 처리하고 청크로 나눕니다
    3. 필요하다면 이전 피드백을 활용해 벡터 인덱스를 미세 조정합니다
    4. 피드백이 반영된 관련성 점수로 검색 및 생성을 수행합니다
    5. 향후 개선을 위해 새로운 사용자 피드백을 수집합니다
    6. 시스템이 점진적으로 학습할 수 있도록 피드백을 저장합니다
    
    Args:
        file_path (str): 처리할 파일 경로
        query (str): 사용자 자연어 쿼리
        feedback_data (List[Dict], optional): 이미 로드된 피드백 데이터, None인 경우 파일에서 로드
        feedback_file (str): 피드백 기록을 저장하는 JSON 파일 경로
        fine_tune (bool): 성공적인 과거 Q&A 쌍을 활용하여 인덱스를 개선할지 여부
        
    Returns:
        Dict: 응답과 검색 메타데이터가 포함된 결과
    """
    # 1. 관련성 조정을 위해 과거 피드백 데이터 로드
    if feedback_data is None:
        feedback_data = load_feedback_data(feedback_file)
        print(f"Loaded {len(feedback_data)} feedback entries from {feedback_file}")
    
    # 2. 문서 처리, 청크 분할 및 임베딩 파이프라인 통과
    chunks, vector_store = process_document(file_path)
    
    # 3. 고품질의 과거 상호작용 결과를 활용하여 벡터 인덱스 미세 조정
    # 이는 성공적인 Q&A 쌍에서 향상된 검색 가능한 콘텐츠를 생성합니다
    if fine_tune and feedback_data:
        vector_store = fine_tune_index(vector_store, chunks, feedback_data)
    
    # 4. 피드백 인식 검색을 포함한 핵심 RAG 실행
    # 참고: 이는 rag_with_feedback_loop 함수에 따라 달라집니다
    result = rag_with_feedback_loop(query, vector_store, feedback_data)
    
    # 5. 향후 성능 향상을 위해 사용자 피드백 수집
    print("\n=== Would you like to provide feedback on this response? ===")
    print("Rate relevance (1-5, with 5 being most relevant):")
    relevance = input()
    
    print("Rate quality (1-5, with 5 being highest quality):")
    quality = input()
    
    print("Any comments? (optional, press Enter to skip)")
    comments = input()
    
    # 6. 피드백을 구조화된 데이터로 포맷팅
    feedback = get_user_feedback(
        query=query,
        response=result["response"],
        relevance=int(relevance),
        quality=int(quality),
        comments=comments
    )
    
    # 7. 피드백을 지속적으로 학습할 수 있도록 저장
    store_feedback(feedback, feedback_file)
    print("Feedback recorded. Thank you!")
    
    return result

## Evaluating Our Feedback Loop

In [ ]:
def evaluate_feedback_loop(file_path, test_queries, reference_answers=None):
    """
    피드백 루프가 RAG 품질에 미치는 영향을, 피드백 적용 전후의 성능을 비교해 평가합니다.
    
    이 함수는 피드백 통합이 검색 및 생성 결과에 어떤 영향을 주는지 실험적으로 측정합니다:
    1. 1차: 모든 테스트 쿼리를 피드백 없이 실행
    2. 기준 답변이 있다면 이를 바탕으로 인위적인 피드백 생성
    3. 2차: 동일한 쿼리를 피드백이 반영된 상태로 실행
    4. 두 결과를 비교해 피드백의 효과를 정량적으로 평가
    
    Args:
        file_path (str): 파일 경로
        test_queries (List[str]): 시스템 성능을 평가하기 위한 테스트 쿼리 목록
        reference_answers (List[str], optional): 평가 및 인위적인 피드백 생성을 위한 기준/골드 스탠다드 답변
        
    Returns:
        Dict: 평가 결과가 포함된 결과:
            - round1_results: 피드백 없는 결과
            - round2_results: 피드백이 반영된 결과
            - comparison: 라운드 간 정량적 비교 지표
    """
    print("=== Evaluating Feedback Loop Impact ===")
    
    # 임시 피드백 파일 생성 (이 평가 세션에만 사용)
    temp_feedback_file = "temp_evaluation_feedback.json"
    
    # 피드백 수집 초기화 (시작 시 비어 있음)
    feedback_data = []
    
    # ----------------------- FIRST EVALUATION ROUND -----------------------
    # 모든 쿼리를 피드백 영향 없이 실행하여 기준 성능 설정
    print("\n=== ROUND 1: NO FEEDBACK ===")
    round1_results = []
    
    for i, query in enumerate(test_queries):
        print(f"\nQuery {i+1}: {query}")
        
        # 문서 처리를 통해 초기 벡터 저장소 생성
        chunks, vector_store = process_document(file_path)
        
        # 피드백 영향 없이 RAG 실행 (비어 있는 피드백 목록)
        result = rag_with_feedback_loop(query, vector_store, [])
        round1_results.append(result)
        
        # 기준 답변이 있다면 인위적인 피드백 생성
        # 이는 시스템 학습을 위한 사용자 피드백을 모방합니다
        if reference_answers and i < len(reference_answers):
            # 기준 답변과의 유사도를 기반으로 인위적인 피드백 점수 계산
            similarity_to_ref = calculate_similarity(result["response"], reference_answers[i])
            # 유사도 (0-1)를 등급 범위 (1-5)로 변환
            relevance = max(1, min(5, int(similarity_to_ref * 5)))
            quality = max(1, min(5, int(similarity_to_ref * 5)))
            
            # 구조화된 피드백 항목 생성
            feedback = get_user_feedback(
                query=query,
                response=result["response"],
                relevance=relevance,
                quality=quality,
                comments=f"Synthetic feedback based on reference similarity: {similarity_to_ref:.2f}"
            )
            
            # 메모리에 추가하고 임시 파일에 저장
            feedback_data.append(feedback)
            store_feedback(feedback, temp_feedback_file)
    
    # ----------------------- SECOND EVALUATION ROUND -----------------------
    # 피드백 통합을 포함하여 동일한 쿼리를 실행하여 개선 측정
    print("\n=== ROUND 2: WITH FEEDBACK ===")
    round2_results = []
    
    # 문서 처리 및 피드백 기반 콘텐츠 활용
    chunks, vector_store = process_document(file_path)
    vector_store = fine_tune_index(vector_store, chunks, feedback_data)
    
    for i, query in enumerate(test_queries):
        print(f"\nQuery {i+1}: {query}")
        
        # 피드백 영향을 포함하여 RAG 실행
        result = rag_with_feedback_loop(query, vector_store, feedback_data)
        round2_results.append(result)
    
    # ----------------------- RESULTS ANALYSIS -----------------------
    # 두 라운드 간 성능 지표 비교
    comparison = compare_results(test_queries, round1_results, round2_results, reference_answers)
    
    # 임시 평가 아티팩트 정리
    if os.path.exists(temp_feedback_file):
        os.remove(temp_feedback_file)
    
    return {
        "round1_results": round1_results,
        "round2_results": round2_results,
        "comparison": comparison
    }

## Helper Functions for Evaluation

In [ ]:
def calculate_similarity(text1, text2):
    """
    두 텍스트 간 의미적 유사도를 임베딩을 사용하여 계산합니다.
    
    Args:
        text1 (str): 첫 번째 텍스트
        text2 (str): 두 번째 텍스트
        
    Returns:
        float: 0과 1 사이의 유사도 점수
    """
    # 두 텍스트의 임베딩 생성
    embedding1 = create_embeddings(embedding_model, [text1], device=device, batch_size=1)[0]
    embedding2 = create_embeddings(embedding_model, [text2], device=device, batch_size=1)[0]
    
    # 임베딩을 numpy 배열로 변환
    vec1 = np.array(embedding1)
    vec2 = np.array(embedding2)
    
    # 두 벡터 간 코사인 유사도 계산
    similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    
    return similarity

In [ ]:
def compare_results(queries, round1_results, round2_results, reference_answers=None):
    """
    Compare results from two rounds of RAG.
    
    Args:
        queries (List[str]): Test queries
        round1_results (List[Dict]): Results from round 1
        round2_results (List[Dict]): Results from round 2
        reference_answers (List[str], optional): Reference answers
        
    Returns:
        str: Comparison analysis
    """
    print("\n=== COMPARING RESULTS ===")
    
    # 시스템 프롬프트
    system_prompt = """You are an expert evaluator of RAG systems. Compare responses from two versions:
        1. Standard RAG: No feedback used
        2. Feedback-enhanced RAG: Uses a feedback loop to improve retrieval

        Analyze which version provides better responses in terms of:
        - Relevance to the query
        - Accuracy of information
        - Completeness
        - Clarity and conciseness
    """

    comparisons = []
    
    # 각 쿼리와 두 라운드의 결과에 대해 반복
    for i, (query, r1, r2) in enumerate(zip(queries, round1_results, round2_results)):
        # 응답 비교를 위한 프롬프트 생성
        comparison_prompt = f"""
        Query: {query}

        Standard RAG Response:
        {r1["response"]}

        Feedback-enhanced RAG Response:
        {r2["response"]}
        """

        # 기준 답변이 있다면 포함
        if reference_answers and i < len(reference_answers):
            comparison_prompt += f"""
            Reference Answer:
            {reference_answers[i]}
            """

        comparison_prompt += """
        Compare these responses and explain which one is better and why.
        Focus specifically on how the feedback loop has (or hasn't) improved the response quality.
        """

        # OpenAI API를 호출하여 비교 분석 생성
        response = client_openai.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": comparison_prompt}
            ],
            temperature=0
        )
        
        # 비교 분석을 결과에 추가
        comparisons.append({
            "query": query,
            "analysis": response.choices[0].message.content
        })
        
        # 각 쿼리에 대한 분석 조각 출력
        print(f"\nQuery {i+1}: {query}")
        print(f"Analysis: {response.choices[0].message.content[:200]}...")
    
    return comparisons

## Evaluation of the feedback loop (Custom Validation Queries)

In [ ]:
import pandas as pd
# 평가 데이터 로드하기
df = pd.read_csv('./data_creation/rag_val_new_post.csv')

# test queries 정의
test_queries = [
    df['query'][0],
    # df['query'][1],
    # df['query'][2],
    # df['query'][3],
    # df['query'][4],
    # df['query'][5],
    # df['query'][6],
    # df['query'][7]
    # ...
]

# 평가를 위한 기준 답변 정의
reference_answers = [
    df['generation_gt'][0],
    # df['generation_gt'][1],
    # df['generation_gt'][2],
    # df['generation_gt'][3],
    # df['generation_gt'][4],
    # df['generation_gt'][5],
    # df['generation_gt'][6],
    # df['generation_gt'][7],
    # ...
    ]


# 파일 경로
file_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

# 평가 실행
evaluation_results = evaluate_feedback_loop(
    file_path=file_path,
    test_queries=test_queries,
    reference_answers=reference_answers
)

In [ ]:
########################################
# # Run a full RAG workflow
########################################

# # Run an interactive example
# print("\n\n=== INTERACTIVE EXAMPLE ===")
# print("Enter your query about AI:")
# user_query = input()

# # Load accumulated feedback
# all_feedback = load_feedback_data()

# # Run full workflow
# result = full_rag_workflow(
#     pdf_path=pdf_path,
#     query=user_query,
#     feedback_data=all_feedback,
#     fine_tune=True
# )

########################################
# # Run a full RAG workflow
########################################